In [1]:
!pip install --upgrade pip
!pip install openai
!pip install z3-solver


In [2]:
!java -version

java version "1.8.0_411"
Java(TM) SE Runtime Environment (build 1.8.0_411-b09)
Java HotSpot(TM) 64-Bit Server VM (build 25.411-b09, mixed mode)


# invaR1ant-evaluation:CODE : Worst-Case Analysis Constraint Generation via LLMs

## Project Overview

This notebook implements an automated framework for leveraging large language models to generate Python generators for constraints that characterize worst-case execution paths in programs. The implementation analyzes how input conditions influence program performance at different input scales by systematically generating and testing constraints.

## Background and Motivation

The `CODE` version of the project addresses a fundamental limitation in directly using LLMs for generating constraints: **token limits**. When working with large input sizes (like N=250, 500, 1000), LLMs often fail to generate complete constraint sets because:

1. The constraint expressions grow quadratically with input size N
2. LLM context windows have fixed token limits (even for advanced models)
3. Hand-crafted constraints become impractical beyond small examples

Instead, this approach uses LLMs to generate **Python functions** that can programmatically produce the complete constraint set for any N value. This "generator pattern" allows us to overcome token limitations while maintaining the LLM's ability to generalize patterns from examples.

## What This Notebook Does

The experiment conducts the following workflow:

1. **Extract Constraints** from existing SMT files as examples (N=1 to N=10)
2. **Query Language Models** (OpenAI API) to generalize these constraints for arbitrary input sizes
3. **Generate SMT-LIB Format Assertions** through Python code generation
4. **Validate Constraints** using Z3 theorem prover
5. **Execute Test Cases** against a Java implementation (SimpleAscendingLast)
6. **Collect and Analyze Results** to verify execution path lengths

## Key Components

- **PromptBuilder**: Extracts constraints from SMT files and builds prompts for OpenAI
- **OpenaiAPI**: Handles communication with OpenAI models with retry logic and rate limiting
- **Services**: Provides utility functions for Z3 solver integration and test execution
- **Experiment**: Orchestrates the end-to-end evaluation process

## Technical Prerequisites

- Python 3.12+ with packages:
  - openai
  - z3-solver
  - dotenv (for environment variable management)
- Java 8+ runtime (JDK 1.8.0_411 tested)
- Z3 SMT solver installed and available in PATH
- OpenAI API key set in environment variables as "OPENAI_KEY"

## Experiment Configuration

The experiment runs with configurable parameters:
- Test input sizes (default: 250, 500, 1000)
- OpenAI model (default: gpt-4o)
- Maximum attempts per testing value (default: 10)
- Temperature for model sampling (configurable)

## Interpreting Results

Results are saved as JSON files containing:
- LLM-generated constraints
- Z3 solver outputs
- Test execution metrics
- Execution path lengths (call counts)

Each test case is considered successful when the actual call count matches the expected count (N+1), confirming the worst-case execution path was triggered.

## Constraint Pattern Analysis

The constraints follow a systematic pattern where:
1. Each input element (in0...inN-1) must be in increasing order
2. The special input value "in" must be greater than all elements of the array
3. The combined constraints create the worst-case execution path in SortedListInsert

## What this notebook does **NOT** do

This notebook does not cover the following aspects:
- **Compiling Java Code**: The Java code has been compiled and is in the `/spf-wca/src/examples/` and  `/spf-wca/build/examples/`directory.
- **Running JPF/SPF on Java Code**: The Java code has been run using JPF and SPF-WCA. The results are in the `/spf-wca/results_own/SimpleAscendingLast/` directory.
- **Compiling and building *JPF-CORE* *JPF-SYMBC* and *SPF-WCA***: The Java PathFinder (JPF) and SPF-WCA have been compiled and built. The JPF-CORE and JPF-SYMBC are in the `/jpf-core/` and `/jpf-symbc/` directories respectively. The SPF-WCA is in the `/spf-wca/` directory.

## Known Limitations

- **Token Limit Workaround**: Although using code generators helps with token limits, the example constraints provided in prompts still grow large with more examples
- **Execution Time**: For large N values, Z3 solving and test execution may take considerable time
- **Model Consistency**: Different OpenAI models may produce varying quality solutions
- **Logging Duplication**: Multiple log handlers may cause duplicate output in Jupyter environments

---

*Note: This notebook is part of research on program analysis and invariant generation using large language models. The approach demonstrates how LLMs can be used to generalize properties of program behavior across different input scales even when constraint expressions exceed token limits.*


In [3]:
import json
import logging
import os
import re
import subprocess
import sys
import time
from contextlib import suppress
from pathlib import Path

import openai
from dotenv import load_dotenv
from openai._types import NotGiven

load_dotenv()

True

In [4]:
class prompt:
    promptStart1 = """I'm experimenting with a program to understand how input conditions influence its worst-case performance, particularly in terms of finding the longest execution path as the input size increases. By conducting a worst-case analysis, I aim to identify constraints that define a valid input set at different input sizes (N). So far, I have found one possible set of correct constraints/conditions (not the only one) that characterize such valid inputs. Here they are:\n\n"""
    promptStart2 = """\n\nGeneralize what makes the set of constraints valid such that we can recover a valid set for N inputs. Provide a Python function that outputs a valid SMT-LIB format assert constraint string for all values of N in SMT-LIB format, matching the format of the examples provided. Ensure the constraints are in canonical form. Don't overfit the data here but also don't oversimplify to the point of trivialness. Make sure none of the given examples contradict your generalization."""
    promptStartTemplate = """Structure your response in the following format and use this code template:\n\n```python"""
    promptfeedbackformat = """Remember respond using the code template. Your format was not correct."""
    promptfeedbackunsat = """Remember respond using the code template. Constraints generated from your function were unsatisfiable."""
    promptfeedbackinvalid = """Remember respond using the code template. Constraints generated from your function were satisfiable but it does not follow the expected path."""
    promptsys = """Return the python function using the code template provided."""


class PromptBuilder:
    def __init__(
        self, constraints_dir="spf-wca/results_own/SimpleAscendingLast/verbose/heuristic", file_prefix="SimpleAscendingLast"
    ) -> None:
        constraints_dir = Path(os.getcwd()) / constraints_dir
        self.constraints_dir = Path(constraints_dir)
        self.file_prefix = file_prefix
        self.constraints = self.__get_constraints()

    def __get_constraints(self) -> str:
        """
        Extract constraints from the SMT files in a sorted order.
        Includes "N=1: none" if no file for N=1 exists, and includes entries up to N=30.
        """
        if not self.constraints_dir.exists():
            logging.error(f"Directory {self.constraints_dir} does not exist.")
            raise FileNotFoundError(f"Directory {self.constraints_dir} does not exist.")

        # Find all SMT files matching the pattern
        smt_files = list(self.constraints_dir.glob(f"{self.file_prefix}_*.smt2"))

        if not smt_files:
            logging.error(f"No SMT files found in {self.constraints_dir}")
            raise FileNotFoundError(f"No SMT files found in {self.constraints_dir}")

        # Define a function to extract N value from the filename
        def extract_n(file_path):
            try:
                n_str = str(file_path).split(self.file_prefix)[-1].split(".smt2")[0]
                return int(n_str.strip("_"))
            except (IndexError, ValueError):
                return float("inf")  # Files that can't be parsed go at the end

        # Create a mapping from N values to their constraints
        constraints_map = {}
        
        # Process each SMT file and extract constraints
        for smt_file in smt_files:
            n = extract_n(smt_file)
            if n == float("inf"):
                continue  # Skip files that don't match the pattern

            with open(smt_file, "r") as file:
                smt_lines = [line.strip() for line in file if line.strip()]
                assertions = [line for line in smt_lines if line.startswith("(assert")]
                assertions_str = "\n".join(assertions)
                constraints_map[n] = assertions_str

        # Determine the maximum N value
        max_n = max(constraints_map.keys()) if constraints_map else 0
        max_n = max(max_n, 30)  # Ensure we go up to at least N=30
        
        # Build the final constraints list with "none" for missing values
        constraints = []
        for n in range(1, max_n + 1):
            if n in constraints_map:
                constraints.append(f"N={n}: {constraints_map[n]}")
            else:
                constraints.append(f"N={n}: None")

        return "\n".join(constraints)

    def build_prompt(self) -> dict:
        """
        Build the prompt for the OpenAI API.
        """
        with open("template.py", "r") as file:
            template = file.read()
        data = prompt()
        return {
            "sys": data.promptsys,
            "start": data.promptStart1
            + self.constraints
            + data.promptStart2
            + data.promptStartTemplate
            + template
            + "\n```",
            "feedback-invalid": data.promptfeedbackinvalid,
            "feedback-unsat": data.promptfeedbackunsat,
            "feedback-format": data.promptfeedbackformat,
        }

In [5]:
class OpenaiAPI:
    """
    OpenAI API interface.
    """

    def __init__(self, model: str, temperature: int = NotGiven()) -> None:
        self.api_key = os.getenv("OPENAI_KEY")
        if not self.api_key:
            msg = "OPENAI_KEY environment variable not set"
            raise ValueError(msg)

        self.model = model
        self.client = openai.OpenAI(api_key=self.api_key)
        self.logger = logging.getLogger(__name__)
        if not self.logger.handlers:
            self.logger.addHandler(logging.StreamHandler(sys.stdout))
        self.logger.setLevel(logging.INFO)
        self.temperature = temperature

    def chat(self, history: list, max_retries=5) -> str:
        """Chat with the OpenAI API with improved error handling."""
        retry_count = 0

        while retry_count < max_retries:
            try:
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=history,
                    temperature=self.temperature,
                )
                return response.choices[0].message.content
            except openai.RateLimitError as e:
                retry_count += 1
                self.logger.warning(f"Rate limit error: {e}")

                if "tokens" in str(e) or "must be reduced" in str(e):
                    self.logger.info("Token limit exceeded - reducing context")
                    history = self.__reduce_context(history)
                    if not history:
                        return "Cannot reduce context further."
                    continue

                wait_time = min(60, 2**retry_count)
                self.logger.info(f"Rate limit exceeded. Waiting {wait_time}s")
                time.sleep(wait_time)
            except openai.BadRequestError as e:
                self.logger.error(f"Bad request: {e}")
                if "maximum context length" in str(e):
                    history = self.__reduce_context(history)
                    if not history:
                        return "Error: Maximum context length exceeded."
                    continue
                return f"Bad request: {str(e)}"
            except (openai.APIConnectionError, openai.APITimeoutError) as e:
                retry_count += 1
                wait_time = min(30, 2**retry_count)
                self.logger.warning(f"Connection error: {e}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
            except Exception as e:
                self.logger.error(f"Unexpected error: {e}")
                return f"Error: {str(e)}"

        return "Maximum retry limit reached"

    def __reduce_context(self, history: list) -> list:
        """
        Reduce the context by 1.
        """
        if len(history) < 3:
            return False

        # Keep prompt and remove second item in the list
        history.pop(2)
        return history

In [6]:
class Services:
    """
    Services for Z3 solver and program execution.
    """

    @classmethod
    def check_z3_installed(cls) -> bool:
        """
        Check if Z3 solver is installed.
        """
        try:
            result = subprocess.run(["z3", "--version"], capture_output=True, text=True)
            return result.returncode == 0
        except FileNotFoundError:
            return False

    @staticmethod
    def Z3check(constants: str, constraints: str) -> dict:
        """
        Check the satisfiability of the SMT2 file and return the test cases.

        Args:
            constants: Constants in the SMT2 file.
            constraints: Constraints in the SMT2 file.

        Returns:
            A dictionary of variable assignments if satisfiable, or an {"error": ...} dict if not.
        """
        logger = logging.getLogger(__name__)
        logger.info("Invoking Z3 solver...")
        if not constants or not constraints:
            error_msg = "Missing constants or constraints"
            logger.error(error_msg)
            return {"error": error_msg}
        smt2 = Services.__format_smt2(constants=constants, constraints=constraints)
        if not smt2:
            error_msg = "SMT2 formatting failed"
            logger.error(error_msg)
            return {"error": error_msg}
        try:
            proc = subprocess.run(
                ["z3", "-in"],
                input=smt2,
                capture_output=True,
                text=True,
                check=False
            )
            output = proc.stdout.strip()
            if "unsat" in output.lower():
                return {"error": "Constraints unsatisfiable: " + output}
            elif "sat" in output.lower():
                model = Services.__parse_z3_model(output, smt2)
                if "error" in model:
                    return model
                return model
            else:
                return {"error": "Unexpected output from Z3: " + output}
        except Exception as e:
            error_msg = f"Error running Z3: {str(e)}"
            logger.error(error_msg)
            return {"error": error_msg}

    @staticmethod
    def __parse_z3_model(output: str, smt2: str) -> dict:
        """Parse Z3 model output into a dictionary."""
        try:
            model_text = output.split("sat", 1)[1].strip()
            if not model_text:
                return {}

            model_dict = {}

            # Find all define-fun blocks
            import re

            pattern = r"\(define-fun\s+([^\s]+)\s+\(\)\s+([^\s]+)\s+(.*?)\)"
            matches = re.findall(pattern, model_text, re.DOTALL)

            for var_name, var_type, var_value in matches:
                value = var_value.strip()
                if var_type == "Int":
                    if value.startswith("(- ") and value.endswith(")"):
                        value = -int(value[3:-1])
                    else:
                        with suppress(ValueError):
                            value = int(value)
                model_dict[var_name] = value

            model_dict["smt2"] = smt2
            model_dict["z3_output"] = output

            return model_dict
        except Exception as e:
            return {"error": f"Error parsing Z3 output: {str(e)}", "raw_output": output}

    @staticmethod
    def __format_smt2(constants: str, constraints: str) -> str:
        """
        Format the SMT2 file with the constants and constraints.
        """
        return f"""
; Allow parallel to speed up solving
(set-option :parallel.enable true)
(set-option :parallel.threads.max 1000)

; Constants
{constants}

; Constraints
{constraints}
(check-sat)
(get-model)
"""

    @staticmethod
    def put_testcase(classpath: str,program: str, *args: list) -> str:
        """
        Put the testcase to the program.

        Args:
            path: Path to the program
            N: Input size
            data: Input data

        Returns:
            Output of the program
        """
        try:
            return subprocess.check_output(["java", "-cp", classpath, program, *args], text=True)
        except subprocess.CalledProcessError as e:
            return f"Error: {e!s}"

    @staticmethod
    def validPython(generalisation: str) -> bool:
        """
        Check if the Python code is valid and contains the expected function and call sequence.

        Args:
            generalisation: Python code.

        Returns:
            True if valid, False otherwise.
        """
        try:
            # Create namespace with mocked input
            namespace = {"input": lambda _: "1"}

            # Compile and execute code
            exec(compile(generalisation, "<string>", "exec"), namespace)

            # Check if function exists
            if "generate_constraints" not in namespace:
                return False

            # Ensure the expected code snippet is present (adjust if necessary)
            expected_snippet = (
                'N = int(input("N="))\nconstraints = generate_constraints(N)\nprint(constraints)'
            )
            if expected_snippet not in generalisation:
                print("Missing the final part of the code")
                return False

            # Test function call
            result = namespace["generate_constraints"](1)
            return bool(result)
        except Exception:
            return False

In [7]:
import logging
import os
import time
from pathlib import Path

# Clear any existing handlers to avoid duplicates
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Create logs directory if it doesn't exist
log_dir = Path(os.getcwd()) / "logs"
log_dir.mkdir(exist_ok=True)

# Create a timestamped log filename
timestamp = time.strftime("%Y%m%d-%H%M%S")
log_file = log_dir / f"experiment_{timestamp}.log"

# Configure both console and file logging
file_handler = logging.FileHandler(log_file)
file_handler.setFormatter(logging.Formatter('%(message)s'))

console_handler = logging.StreamHandler()
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

# Set up logging with both handlers
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[console_handler, file_handler],
    force=True
)

print(f"Logging output to: {log_file}")

class Experiment:
    """
    Experiment class that:
     - Builds prompts for various input sizes.
     - Queries the OpenAI API to get SMT-LIB constraints.
     - Uses the Z3 solver to produce concrete test cases.
     - Records the results.
    """

    def __init__(self, model: str, testing_values=None, max_attempts=3, temperature=NotGiven()) -> None:
        self.logger = logging.getLogger(__name__)
        self.prompt_builder = PromptBuilder()
        self.constraints = self.prompt_builder.constraints
        self.max_attempts = max_attempts
        self.testing_values = testing_values or [250, 500, 1000]
        self.model = model

        # Check Z3 installation before proceeding
        if not Services.check_z3_installed():
            raise RuntimeError("Z3 solver not found in PATH. Please install Z3.")

        try:
            self.openai_api = OpenaiAPI(model, temperature=temperature)
        except ValueError as e:
            raise RuntimeError(f"Failed to initialize OpenAI API: {e}")

        self.results = {}

    def __extract_function(self, text: str) -> str:
        """
        Extract the function from the OpenAI response.
        """
        regex = r"```python(.*)```"
        match = re.search(regex, text, re.DOTALL)
        return match.group(1).strip() if match else None

    def run(self, save_results=True) -> dict:
        start_time = time.time()

        for testing_value in self.testing_values:
            self.logger.info(f"Testing with N = {testing_value}")
            history = []
            prompt_data = self.prompt_builder.build_prompt()
            history.append({"role": "system", "content": prompt_data["sys"]})
            history.append({"role": "user", "content": prompt_data["start"]})
            for attempt in range(1, self.max_attempts + 1):
                self.logger.info(f"Attempt {attempt} of {self.max_attempts}")
                self.logger.info(f"Prompt:\n{history}")
                response = self.openai_api.chat(history)
                self.logger.info(f"OpenAI Response:\n{response}")
                history.append({"role": "assistant", "content": response})
                function_code = self.__extract_function(response)

                # Check if the response contains a Python function code block
                if not function_code:
                    self.logger.warning("No Python function code block found. Retrying...")
                    history.append({"role": "user", "content": prompt_data["feedback-format"]})
                    continue

                # Check if the Python code is valid and follows expected structure
                if not Services.validPython(function_code):
                    self.logger.warning("Invalid Python function code. Retrying...")
                    history.append({"role": "user", "content": prompt_data["feedback-invalid"]})
                    continue

                try:
                    namespace = {"input": lambda _: "1"}
                    exec(compile(function_code, "<string>", "exec"), namespace)
                    if "generate_constraints" not in namespace:
                        self.logger.warning("Function generate_constraints not found. Retrying...")
                        history.append({"role": "user", "content": prompt_data["feedback-invalid"]})
                        continue

                    constraints = namespace["generate_constraints"](testing_value)
                    if not constraints:
                        self.logger.warning("Generated constraints are empty. Retrying...")
                        history.append({"role": "user", "content": prompt_data["feedback-invalid"]})
                        continue

                    constants = "\n".join([f"(declare-const in{i} Int)" for i in range(testing_value)])
                    constants += "\n(declare-const in Int)"

                    # Check constraints with Z3
                    z3_result = Services.Z3check(constants, constraints)

                    if not z3_result or 'unsat' in z3_result:
                        self.logger.warning("Constraints are unsatisfiable. Retrying...")
                        history.append({"role": "user", "content": prompt_data["feedback-unsat"]})
                        continue

                    if "error" in z3_result:
                        self.logger.warning(f"Z3 error: {z3_result['error']}. Retrying...")
                        history.append(
                            {
                                "role": "user",
                                "content": prompt_data["feedback-invalid"] + f"\n{z3_result['error']}",
                            }
                        )
                        continue

                    self.results[testing_value] = {
                        "Success": False,
                        "Z3": z3_result,
                        "Function": function_code,
                        "OpenAI": response,
                        "Attempts": attempt,
                        "Constraints": constraints,
                    }

                    self.logger.info(f"Executing test with N={testing_value}")
                    class_path = str(Path(os.getcwd()) / "test")
                    program = "SimpleAscendingLast"

                    try:
                        output = Services.put_testcase(
                            classpath=class_path,
                            program=program,
                            *list(z3_result.values())
                        )
                        output_value = output.strip()
                        success = "true" in output_value.lower()
                        self.results[testing_value]["Success"] = success
                        self.results[testing_value]["Output"] = output

                        if success:
                            self.logger.info("Success!")
                            break
                        self.logger.warning(
                            f"Final loop condition not met for N={testing_value}. Retrying..."
                        )
                    except Exception as e:
                        self.logger.error(f"Error executing test: {e}")
                        self.results[testing_value]["Error"] = f"Execution error: {str(e)}"
                except Exception as e:
                    self.logger.error(f"Error executing function: {e}")
                    history.append({"role": "user", "content": prompt_data["feedback-invalid"]})
            else:
                self.logger.error(f"Failed after {self.max_attempts} attempts for N={testing_value}")
                self.results[testing_value] = {"Success": False, "Error": f"Failed after {self.max_attempts} attempts"}

        if save_results:
            self.save_results()

        duration = time.time() - start_time
        self.logger.info(f"Experiment completed in {duration:.2f} seconds")
        return self.results

    def save_results(self, filename=None):
        """Save results to a JSON file."""
        if not filename:
            timestamp = time.strftime(f"%Y%m%d-%H%M%S")
            filename = f"{self.model}_{timestamp}.json"

        try:
            with open(filename, "w") as f:
                json.dump(self.results, f, indent=2)
            self.logger.info(f"Results saved to {filename}")
        except Exception as e:
            self.logger.error(f"Failed to save results: {e}")

Logging output to: /Users/dankoh/invaR1ant-evaluation/logs/experiment_20250314-155416.log


In [8]:
experiment = Experiment(
    "gpt-4o",
    testing_values=[100,250]

    )
experiment.run()

Testing with N = 100


2025-03-14 15:54:24,242 - __main__ - INFO - Testing with N = 100


Attempt 1 of 3


2025-03-14 15:54:24,243 - __main__ - INFO - Attempt 1 of 3


Prompt:
[{'role': 'system', 'content': 'Return the python function using the code template provided.'}, {'role': 'user', 'content': 'I\'m experimenting with a program to understand how input conditions influence its worst-case performance, particularly in terms of finding the longest execution path as the input size increases. By conducting a worst-case analysis, I aim to identify constraints that define a valid input set at different input sizes (N). So far, I have found one possible set of correct constraints/conditions (not the only one) that characterize such valid inputs. Here they are:\n\nN=1: None\nN=2: (assert  ( <  in1 in0))\nN=3: (assert (and  ( <  in0 in1)  ( <  in2 in0)))\nN=4: (assert (and (and  ( <  in0 in1)  ( <  in1 in2))  ( <  in3 in0)))\nN=5: (assert (and (and (and  ( <  in0 in1)  ( <  in1 in2))  ( <  in2 in3))  ( <  in4 in0)))\nN=6: (assert (and (and (and (and  ( <  in0 in1)  ( <  in1 in2))  ( <  in2 in3))  ( <  in3 in4))  ( <  in5 in0)))\nN=7: (assert (and (and (and

...

Results saved to gpt-4o_20250314-155525.json


2025-03-14 15:55:25,622 - __main__ - INFO - Results saved to gpt-4o_20250314-155525.json


Experiment completed in 61.38 seconds


2025-03-14 15:55:25,623 - __main__ - INFO - Experiment completed in 61.38 seconds


{100: {'Success': False, 'Error': 'Failed after 3 attempts'},
 250: {'Success': False, 'Error': 'Failed after 3 attempts'}}